In [ ]:
import h5py as h
import numpy as np
import pandas as pd

import skimage.morphology as morph
import skimage.measure as measure

import os
import skan

def lengths_3d(array, small_mito=100):
    # remove small particles
    small_rem = morph.remove_small_objects(morph.label(array),min_size=small_mito)
    array = np.array(small_rem > 0) * 1

    # skeletonize image
    skel = morph.skeletonize(array, method='lee')
    #fat_skel = morph.binary_dilation(skel)

    # label skeleton per branch
    labs2 = skan.Skeleton(skel).path_label_image()
    #fat_labs = skan.Skeleton(fat_skel).path_label_image()

    # get data of the skeletons, spacing = dimensions of the image.
    branch_data = skan.summarize(skan.Skeleton(skel,spacing=(0.2,0.045,0.045)))

    # make df
    # calculate average branch length per object, max branch length and total length
    av_branch = branch_data.groupby('skeleton-id').mean()['branch-distance'].tolist()
    max_branch = branch_data.groupby('skeleton-id').max()['branch-distance'].tolist()
    n_branch = branch_data.groupby('skeleton-id').count()['branch-distance'].tolist()
    total_length = branch_data.groupby('skeleton-id').sum()['branch-distance'].tolist()

    return pd.DataFrame({'av_branch_length':av_branch,'max_branch_length':max_branch,'n_branches':n_branch,'total_mito_length':total_length})

# folder with segmentations from Ilastik
dir = r"X:\Kleele\Nov_2024\02-Analysis\241108_RPE1_drug_treatments_screen\Ilastik_segmentation"

# folder with cell segmentations
cell_dir = r"X:\Kleele\Nov_2024\02-Analysis\241108_RPE1_drug_treatments_screen\cell_masks"

# set threshold for smallest mitochondria possible
small_mito = 100

imgs = os.listdir(dir)

res = []
for file in os.listdir(dir):
    if file.endswith('tation.h5'):
        basename = file.split('_decon')[0] + '_decon'
        
        mitos = h.File(os.path.join(dir, file))['exported_data'][:,:,:,0]

        mitos[mitos == 2] = 0

        # load cell segmentation 
        cell_seg = np.load(os.path.join(cell_dir, basename + '_cell_mask.npy'))
        cell_seg = np.repeat(cell_seg[np.newaxis,:,:], mitos.shape[0], axis=0)

        # check if there is a cell
        if cell_seg.max() > 0:

            for cell in range(1,cell_seg.max()+1):
                # set everything outside the cell volume to 0
                temp_mito = mitos.copy()
                temp_mito[cell_seg != cell] = 0


                temp = lengths_3d(temp_mito)
                temp['image'] = basename
                temp['cell'] = cell
                res.append(temp)

res = pd.concat(res)
res.to_csv(os.path.join(dir, 'mito_lengths.csv'), index=False)


In [ ]:
# area / form factor calculation 
def calculate_area_form(array, dimensions=(0.2, 0.045, 0.045), small_mito=100):
    props = ['area','eccentricity','axis_major_length', 'axis_minor_length', 'bbox', 'centroid']
    
    # label array and remove small particles
    small_rem = morph.remove_small_objects(morph.label(array),min_size=small_mito)
    array = np.array(small_rem > 0) * 1

    # label
    lab = measure.label(array)

    # calculate regionprops
    df = pd.DataFrame(measure.regionprops_table(lab, properties=props))

    # convert to right dimensions
    df['area'] = df['area'] * dimensions[0] * dimensions[1] * dimensions[2]
    df['axis_major_length'] = df['axis_major_length'] * dimensions[1]
    df['axis_minor_length'] = df['axis_minor_length'] * dimensions[1]
    
    return df


res = []
for file in os.listdir(dir):
    if file.endswith('tation.h5'):
        basename = file.split('_mito')[0]
        
        mitos = h.File(os.path.join(dir, file))['exported_data'][:,:,:,0]

        mitos[mitos == 2] = 0

        # load cell segmentation 
        cell_seg = np.load(os.path.join(cell_dir, basename + '_mito_cell_mask.npy'))
        cell_seg = np.repeat(cell_seg[np.newaxis,:,:], mitos.shape[0], axis=0)

        if cell_seg.max() > 0:

            for cell in range(1,cell_seg.max()+1):
                # set everything outside the cell volume to 0
                temp_mito = mitos.copy()
                temp_mito[cell_seg != cell] = 0


                temp = calculate_area_form(temp_mito)
                temp['image'] = basename
                temp['cell'] = cell
                res.append(temp)

res = pd.concat(res)
res.to_csv(os.path.join(dir, 'mito_shape.csv'), index=False)

In [ ]:
# check what happens with napari. 
import glob 
import napari
from skimage.io import imread

orig_dir = r"B:\Kleele\Nov_2024\01-Data\241107_RPE1_TOM20staygold_drug_treatments"

i = 0

file = glob.glob(os.path.join(dir, '*tation.h5'))[i]

mitos = h.File(os.path.join(dir, file))['exported_data'][:,:,:,0]

orig = imread(os.path.join(orig_dir, file.split('_MM')[0], file.split('_mito')[0] + '.tif'))

# label array and remove small particles
small_mito = 100
small_rem = morph.remove_small_objects(morph.label(mitos),min_size=small_mito)
small_rem = np.array(small_rem > 0) * 1

# label
lab = measure.label(small_rem)

# skeleton 
skel = morph.skeletonize(lab, method='lee')

# label skeletons
labs2 = skan.Skeleton(skel).path_label_image()


# view all of them in napari 
viewer = napari.view_image(orig, name='original', blending='additive', colormap='grey')
viewer.add_labels(mitos, name='Ilastik_segmentation')
viewer.add_labels(small_rem, name='small mitos removed')
viewer.add_labels(lab, name='mitos labeled')
viewer.add_labels(skel, name='skeletons')
viewer.add_labels(labs2, name='skeletons labeled')